# [2021 gnssrefl short course](https://www.unavco.org/event/2021-gnss-interferometric-reflectometry/)

---

## Homework 1

**Due date:** This homework is to be completed **before** the short course given on October 21. You need to make
sure the software has been properly installed and you have successfully completed the "homework 0" assignment.

**Purpose:** Learn how to set your azimuth and elevation angle mask

In [ ]:
import os
import sys
import re
import json
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import run_gnssrefl 
import gnssrefl_helpers

%matplotlib inline

In [ ]:
#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment()
else:
     print('environment variable ORBITS path is', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is', os.environ['EXE'])
        
refl_code_loc = os.environ['REFL_CODE']
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
gnssrefl_helpers.download_crx2rnx()

The purpose of this homework is to get you to better understand how to properly run gnssrefl. Each GNSS station is different and you will have to decide which areas around the antenna you want to measure. This is sometimes called setting the azimuth and elevation angle mask. The [gnss-reflections.org](https://gnss-reflections.org) webapp has some tools we will utilize to help you determine this mask.

# Example 1

Lets first start with looking at the station **at01** in st. Micheal, Alaska.

The data from this GNSS site will be used to measure sea level, so the relevant reflecting surface is the ocean. 

<img src="../../data/geoid-at01.png" width="400">

Run the cell below to view the geoid functionality in the gnss-reflections webapp. Using this, we will get an idea of its surroundings. You can enter the station coordinates by hand if you know them, but since at01 is part of a public archive known to geodesists, coordinates have been stored in the webapp. Just type in at01 for the station name.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="1000" height="600"></iframe>

You can see the general location of the antenna with respect to the coast. Make a note of its height with respect to mean sea level.


Next use the ReflZones tab of the webapp to try out different elevation angle and azimuth angle masks. Make a note of the angles that you think will give you a good reflection off the water.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones" width="1000" height="700"></iframe>

Each one of the yellow/blue/red clusters represents the reflection zone for a single rising or setting GPS satellite arc. The colors represent different elevation angles - so yellow is lowest (5 degrees), blue (10 degrees) and so on.
Change to the different elevation angles to get a better idea of what they look like (use the 'Return to the Reflection Zone API' button to return to the prvious page to re-set different values).

For this site we would want to be measuring water levels. We can see very clearly that 0 to 360 degress would capture part of the land for the station. Go back and change the angles until we are only covering area that contains water. What azimuth angles did you choose?

By default <code>rinex2snr</code> tries to find the RINEX data for you by looking at a few
key archives.  However, if you know where the data are, it will be faster to specify it. For this example we will use the UNAVCO archive.

In [ ]:
# Download SNR 
# we are going to just pick a year and day
# this will generate a SNR file at a sampling rate of 15 seconds.
station = 'at01'
year=2020
doy=109

run_gnssrefl.rinex2snr(station=station, year=year, doy=doy, archive='unavco')

Now we can use the `quicklook` function to give you a visual assessment of the reflection characteristics of your GNSS site. It is not meant for routine analysis of your data - instead it is meant to give you a better understanding of how to choose settings for that routine analysis, which is done in gnssir. Using this should confirm some of values we have so far gotten an idea of using the web apps.

quickLook takes all the SNR data at a site and splits it into four geographic quadrants (northwest, northeast, southwest, southeast). Within those quadrants, it identifies rising and setting arcs for one GNSS transmitter frequency and uses a periodogram to estimate the dominant Reflector Height (RH) in meters. Each color represents a different satellite.

The y-axis is the spectral amplitude in converted SNR units (volts/volts). If you see a strong peak in the periodogram, that means you will have a good estimate of the RH for that satellite arc. The data represented in gray are "failed" periodograms.


The second plot returned is a summary for various quality control metrics.

* the top plot is the color coded RH retrieval color: blue for good and gray for bad. What you are looking for are RH that are consistent with your site. In this case the antenna is about 12 meters above sea level.

* the middle plot is the "peak to noise ratio" which is defined as peak spectral amplitude divided by the average spectral value over a defined noise region. In quickLook this will be the same region you used for your RH estimate. This value is surface dependent. If you happen to check the peak to noise ratio on a day at peak vegetation growth, you might be fooled into thinking the site did not work. Similarly, a windy day on a lake will produce lower peak to noise ratios than a day without wind. I generally use 3.5 for snow, 3.2 for lakes, and 2.7 for tides.

* the bottom plot is the spectral amplitude for the RH peak. This will depend -again - on the surface type. Different surfaces (water, ice, snow, bare soil) reflect signals differently. It also reflects how smooth the surface is and which elevation angle limits were used.

Now, lets run quicklook for at01, year 2020 and day of year 109

In [ ]:
# the pltscreen parameter we will set to True to show the plotted data. 
# The data is also returned so we can plot ourselves - this will be shown in the next homework.
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, pltscreen=True)

Looking at the plots we just generated, what went wrong? Nearly every single retrieval is set as bad (i.e. it is gray rather than blue).

It's important to remember that when we run quicklook like did above, that we didn't change any of the defaults. Lets take a look and see what the defaults are:

In [ ]:
run_gnssrefl.quicklook?

Now use your knowledge of the site's height above mean sea level to properly window the reflector height and using the azimuth and elevation angle mask you picked out earlier. Do you think the retrievals are better now? Should your azimuth mask be modified?

**Help**
<details>
<summary>Click here if you need a hint</summary>
try height between 8 and 15, and degrees elevation angles 5 and 13
</details>

In [ ]:
height_min = 
height_max = 

elevation_angle_min = 
elevation_angle_max = 

values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, 
                                  h1=height_min, h2=height_max,
                                  e1=elevation_angle_min, e2=elevation_angle_max,
                                  pltscreen=True)

Now you see strong retrievals in the Lomb Scargle periodograms with the reflector height of about 12 meters, as expected.
You also see good retrievals at azimuths sweeping from true north to about 220 degrees. Was this close to your orginial best azimuth range estimate? 

We will see in later homeworks how to move forward from here to actually run gnssir and get results. For now, lets try again with another station.

# Example 2

**tgho** is a station in New Zealand operated by [GNS](https://www.gns.cri.nz).The GNSS site is located on a platform in Lake Taupo. It records standard GPS and Glonass signals at a low sample rate (30 sec).
<img src="../../data/tgho_barker.jpeg" width="400">

Lets look at the geoid functionality on the web application:

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="1000" height="600"></iframe>

and the reflection zone functionality on the web app:

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones" width="1000" height="700"></iframe>

We can tell that just by looking at the google maps from the webapp that it won't be as easy to figure out the azimuth range (or elevation angle) range. Its height above mean sea level is given, but that is not the distance to the water shown in the photo. We will instead use quicklook for some feedback.

In [ ]:
station = 'tgho'
# picking a year and day of year
year=2020
doy=300

# get snr (let's use GFZ orbits, which is multi-GNSS setting orb='gnss')
run_gnssrefl.rinex2snr(station=station, year=year, doy=doy, orb='gnss', archive='nz')

# run quicklook with the defaults
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, pltscreen=True)

What do you think the vertical distance is between the antenna and the water? Using the default elevation angles is likely combining water and pier reflections. Try retricting the close reflections (from the pier) by using a larger lower bound. Also compare the defaults with 5-15 degrees elevation angles to see if this improves the retrievals.

In [ ]:
min_height =
max_height =
min_elevation_angle = 
max_elevation_angle = 

**Help**
<details>
<summary>Click here if you need a hint</summary>
try height between 2 and 8
</details>

In [1]:
values, metrics = run_gnssrefl.quicklook(station, year, doy=doy, pltscreen=True,
                                         h1=min_height, h2=max_height,
                                         e1=min_elevation_angle, e2=max_elevation_angle)

NameError: name 'run_gnssrefl' is not defined

Refer to this [publication](https://www.kristinelarson.net/wp-content/uploads/2021/05/Holden-May2021.pdf) on lake Taupo for more information on this site.

We will see in the next homeworks how to set the azimuth and elevation angle values when running gnssir analysis.